# Cubic Bezier Spline

- Definition with lens constraints: C1 or G1 continuity, tangent at 0 = 0
- Line intersection with Newton's method accross all intervals, differentiable with no grad trick
- Normal at the intersection
- Resampling (aka splitting)

=> possibly use sigmoid function to constraint Cx in [0, 1], in the interval between knot?
=> or just positive constraint?


Parametrization:
    r: radius
    n: number of intervals (min 1)
    X: knots X values = linspace(0, r)
    Y: knots Y values
    C: control points

    X.shape = n+1 | all values fixed
    Y.shape = n parameters | (+first value implicitly 0)
    CX.shape = n+1 parameters
    CY.shape = n parameters | (+first value implicitly 0)

    for origin at radius, translate all points by y value of curve at radius

    t: bezier parameter, from 0 to n


evaluation at t:
    integer part of t gives the interval, and so relevant knots and control points
    decimal part of t gives the parameters for matrix form of bezier curve

normal at t:
    like evaluation but use derivative matrix
    rotate pi/2 with [-y, x]

resample:
    compute new knots by evaluating at the new X
    compute new control points by interpolation

intersection:
    newton's method over the piecewise defined equation of the intersection with line ax+by+c=0


In [ ]:
import torch

from torchlensmaker.shapes import BezierSpline
from torchlensmaker.shapes.common import intersect_newton, mirror_points
        


import matplotlib.pyplot as plt

def render_spline(ax, spline):
    X, Y, CX, CY = spline.coefficients()
    ax.scatter(X.detach().numpy(), Y.detach().numpy(), color="steelblue", marker="x")
    ax.scatter(CX.detach().numpy(), CY.detach().numpy(), color="#999999", marker=4)

    next_cp = torch.stack((CX, CY), dim=-1)
    next_knot = torch.stack((X, Y), dim=-1)
    mirrors = mirror_points(next_cp, next_knot)
    ax.scatter(mirrors[:, 0].detach().numpy(), mirrors[:, 1].detach().numpy(), color="#999999", marker=5)

    sampleTs = torch.linspace(-spline.num_intervals, spline.num_intervals, 50)
    points = spline.evaluate(sampleTs).detach().numpy()
    ax.plot(points[:, 0], points[:, 1], color="steelblue")

    normalst = torch.linspace(-spline.num_intervals, spline.num_intervals, 11)
    normals_origins = spline.evaluate(normalst).detach().numpy()
    normals_vectors = spline.normal(normalst).detach().numpy()
    for o, n in zip(normals_origins, normals_vectors):
        ax.plot([o[0], o[0]+n[0]], [o[1], o[1]+n[1]], color="grey", linestyle="--")

    ax.set_aspect("equal")


def render_lines(ax, lines):
    ymin, ymax = ax.get_ylim()
    for line in lines:
        a, b, c = line
        x0 = (-c-b*ymin)/a
        x1 = (-c-b*ymax)/a
        ax.plot([x0, x1], [ymin, ymax], color="orange")

def render_points(ax, points):
    if isinstance(points, torch.Tensor):
        points = points.detach().numpy()
    ax.scatter(points[:, 0], points[:, 1], color="green", marker="o")


def test():
    Y = torch.tensor([-2.7340])
    CX = torch.tensor([1.23, 9.])
    CY = torch.tensor([-4.5])

    spline = BezierSpline(7.27, (Y, CX, CY))
    
    spline.dump()
    lines = torch.tensor([
        [1., 0.5, 15.],      # no intersection
        [-9.0, -1.5, -25.],  # 1st intersect
        [3.6, -5, -20],      # 2nd intersect
        [3., 0., -18.],      # 3rd intersect (vertical)
    ])

    fig, ax = plt.subplots(figsize=(12, 6))                
    render_spline(ax, spline)
    render_lines(ax, lines)

    sols = intersect_newton(spline, lines)

    # Keep only valid solutions (within domain)
    valid = torch.logical_and(sols <= spline.domain()[1], sols >= spline.domain()[0])
    sols = sols[valid]
  
    collisions = spline.evaluate(sols)
    render_points(ax, collisions)
    plt.show()


    for i in range(3):
        spline = spline.wiggle(cx=0.05, cy=0.05, y=0.1)
        spline = spline.resample()
        fig, ax = plt.subplots(figsize=(12, 6))
        render_spline(ax, spline)
        render_lines(ax, lines)
        sols = intersect_newton(spline, lines)
        
        # Keep only valid solutions (within domain)
        valid = torch.logical_and(sols <= spline.domain()[1], sols >= spline.domain()[0])
        sols = sols[valid]
        
        collisions = spline.evaluate(sols)
        render_points(ax, collisions)
        plt.show()
    

test()

In [ ]:


def test2():
    lens_width = 15.0
    #spline = BezierSpline(width=lens_width, init=([-3.0], [0.2*lens_width, 1.2*lens_width], [-4.8]))
    spline = BezierSpline(width=lens_width, init=([5.1431], [7.9189, 13.8972], [3.6509]))

    lines = torch.tensor([[  1.0000,  -0.5000,  13.4000],
        [  1.0000,  -0.0000,  10.4222],
        [  1.0000,  -0.0000,   1.4889],
        [  1.0000,  -0.0000,  -1.4889],
        [  1.0000,  -0.0000, -10.4222],
        [  1.0000,  -0.0000, -13.4000]])

    fig, ax = plt.subplots(figsize=(12, 6))                
    render_spline(ax, spline)
    render_lines(ax, lines)

    sols = intersect_newton(spline, lines)
    collisions = spline.evaluate(sols)
    render_points(ax, collisions)
    plt.show()
    

test2()